In [127]:
# https://github.com/Protean-Labs/subgrounds
from subgrounds.subgrounds import Subgrounds
from subgrounds.subgraph import SyntheticField
import pandas as pd
from datetime import date, datetime, timedelta
from operator import add

In [128]:
sg = Subgrounds()
snapshot = sg.load_api('https://hub.snapshot.org/graphql')

In [129]:
snapshot.Vote.date_vote = SyntheticField(
    f = lambda timestamp: str(datetime.fromtimestamp(timestamp)),
    type_ = SyntheticField.STRING,
    deps = snapshot.Vote.created
)

snapshot.Vote.voter_choice = SyntheticField(
    f = lambda x: 'Authorize Fraxswap pool' if x == 1 else 'Do not authorize',
    type_ = SyntheticField.STRING,
    deps = snapshot.Vote.choice
)

proposal = snapshot.Query.proposal(
    id = '0x721887691ff47e6eec89f1be98013444c8ffd2d38cf2bc477647ff79506d5bae'
)

votes = snapshot.Query.votes(
    first = 124,
    orderBy = 'created',
    orderDirection = 'asc',
    where = {
        'proposal': '0x721887691ff47e6eec89f1be98013444c8ffd2d38cf2bc477647ff79506d5bae'
    }
)

In [130]:
proposal_scores_strat = sg.query_json([
    proposal.scores_by_strategy
    ])

scores_by_strategy = list(map(
    add, 
    proposal_scores_strat[0]['x313efb831bc47293']['scores_by_strategy'][0], 
    proposal_scores_strat[0]['x313efb831bc47293']['scores_by_strategy'][1]
    )
)
strat_names = ['sOHM v1', 'OHM-DAI SLP', 'sOHM v2', 'sOHM v3', 'sOHM Multichain LP', 'gOHM']

proposal_scores_by_strat = pd.DataFrame(
    data= {'votes': scores_by_strategy, 'asset': strat_names}, 
    columns= ['asset','votes'])
proposal_scores_by_strat

,asset,votes
0,sOHM v1,2.800000e-08
1,OHM-DAI SLP,0.000000e+00
2,sOHM v2,6.770698e+00
3,sOHM v3,5.590144e+03
4,sOHM Multichain LP,4.706992e+03
5,gOHM,1.193595e+05


In [147]:
proposal_votes = sg.query_df([
    votes.voter,
    votes.date_vote,
    votes.voter_choice,
    votes.vp
])

vp_by_strategy_json = sg.query_json([votes.vp_by_strategy])[0]['x2daa84da485911ce']
vp_by_strategy = {
    'sOHM v1': [], 
    'OHM-DAI SLP': [], 
    'sOHM v2': [], 
    'sOHM v3': [], 
    'sOHM Multichain LP': [], 
    'gOHM': []
}

for i in range(0, len(vp_by_strategy_json)):
    vp_by_strategy['sOHM v1'].append(vp_by_strategy_json[i]['vp_by_strategy'][0]),
    vp_by_strategy['OHM-DAI SLP'].append(vp_by_strategy_json[i]['vp_by_strategy'][1]),
    vp_by_strategy['sOHM v2'].append(vp_by_strategy_json[i]['vp_by_strategy'][2]),
    vp_by_strategy['sOHM v3'].append(vp_by_strategy_json[i]['vp_by_strategy'][3]),
    vp_by_strategy['sOHM Multichain LP'].append(vp_by_strategy_json[i]['vp_by_strategy'][4]),
    vp_by_strategy['gOHM'].append(vp_by_strategy_json[i]['vp_by_strategy'][5])

for key in vp_by_strategy.keys():
    proposal_votes[key] = vp_by_strategy[key]

proposal_votes.head()

,votes_voter,votes_date_vote,votes_voter_choice,votes_vp,sOHM v1,OHM-DAI SLP,sOHM v2,sOHM v3,sOHM Multichain LP,gOHM
0,0xC71deD6f0E5660DFe79C849058Afd9BE7a3c85b8,2022-04-29 21:17:34,Authorize Fraxswap pool,2136.032186,0.0,0,0.0,0.0,0.0,2136.032186
1,0xaFFc125BeDcEeB034110c1939426A3948791ef18,2022-04-29 21:26:25,Authorize Fraxswap pool,3.951647,0.0,0,0.0,0.0,0.0,3.951647
2,0x0D389aa79159030f4c7651eA98F56193f448C3F9,2022-04-29 21:30:41,Authorize Fraxswap pool,7.815373,0.0,0,0.0,0.0,0.0,7.815373
3,0x6C3Ae4117107a33505A83fD7D5ef4c103De52631,2022-04-29 21:40:04,Authorize Fraxswap pool,206.380899,0.0,0,0.0,0.0,0.0,206.380899
4,0x9b0E7954e36F9262DDe23f46624D746Be965C234,2022-04-29 22:04:59,Authorize Fraxswap pool,1.539038,0.0,0,0.0,0.0,0.0,1.539038
